## Applications - Managing a company's inventory

This lecture provides a framework for managing inventory in a manufacturing based company (i.e. one that produces material goods such as soap, food, or chemicals).

---

# Table of Contents


#### <a href='#1'>A Primer on Inventory</a>
#### <a href='#2'>A primer on product demand</a>
#### <a href='#7'>The accounting nuts & bolts</a>
#### <a href='#3'>Data Manipulation </a>
#### <a href='#4'>A note on Weeks of Supply</a>
#### <a href='#5'>A note on production planning</a>
#### <a href='#6'>Preliminary Results & Commentary</a>
#### <a href='#6'>Common Inventory Issues that may throw a wrench in things</a>


<a id='1'></a>
## A primer on inventory.

Inventory is a term accountants use to describe the assets on hand that a company intends to sell as a part of its primary business activity.  For example, when a box factory sells boxes to a meat packer, it has sold its inventory.  Conversely, when that same box factory sells off some of it's receivables (I.O.U.s from other companies) it is NOT selling its inventory.  

For most manufacturing based businesses, inventory typically ties up more capital throughout a calendar year than any other asset with the exception of PP&E (property, plant & equipment), so most of these companies have a huge interest in managing it efficiently.  In this lecture, we will examine a hypothical company's current inventory position, discuss why it may be good or bad, propose a possible path for improvement, and conclude with a discussion on why inventory optimization is a critical component to the success of modern companies.  



In [ ]:
#As always, import all needed libraries first
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import numpy as np

warnings.filterwarnings('ignore')

In [ ]:
#Read in inventory data
inventory_data = pd.read_csv('Inventory_Data.csv')

In [ ]:
#Get a feel for the structure of the data
inventory_data.head()

<a id='2'></a>
## A primer on product demand
When attempting to forecast inventory, there are two big forces that will ultimately drive how much a company will be able and desire to have on hand.  The first force is production capabilities.  For the purposes of this lecture we will assume that the manufacturing sites are built to adequatly produce any needed amount of inventory, and that raw materials are not in short supply. 

The other driving force is product demand.  If people are only buying a few hundred units from a company, it would be silly to manufacture millions.  On the other hand, if demand for the product is going off the charts, a company will need to make sure it sustains enough inventory to satisfy its customers, and not miss out on potential sales. Companies will typically dedicate huge amounts of resources to study market sentiment and guage what product demand will be.  For this lecture, a hypothetical demand forecast is provided.   

In [ ]:
#Read in demand forecast
demand_data = pd.read_csv('Demand_Plan.csv')

In [ ]:
#Get a feel for the structure of the data
demand_data.head()

When working with seperate sets of data, a great thing to start with is to see what relationships the two data sets have.  In the above two data sets, we can see that both data sets show a warehouse, SKU, and product family.  This will be important when merging the two sets for analysis.

<a id='7'></a>
## The accounting nuts and Bolts
1. Inventory is the raw materials used to produce goods as well as the goods that are available for sale.
![image.png](Images/rm.jpg)
2. It is classified as a current asset on a company's balance sheet.
![image.png](Images/bs.jpg)
3. The three types of inventory include raw materials, work-in-progress, and finished goods. 
4. Inventory is valued in one of three ways, including the first-in, first-out method; the last-in, first-out method; and the    weighted average method.
![image.png](Images/val.jpg)
5. Inventory management allows businesses to minimize inventory costs as they create or receive goods on an as-needed basis.
![image.png](Images/opt.jpg)

## FIFO
![image.png](Images/fifo.jpg)
1. First In, First Out (FIFO) is an accounting method in which assets purchased or acquired first are disposed of first.
2. FIFO assumes that the remaining inventory consists of items purchased last.
3. Often, in an inflationary market, lower, older costs are assigned to the cost of goods sold under the FIFO method, which results in a higher net income than if LIFO were used.

## LIFO
![image.png](Images/lifo.jpg)
1. Last in, first out (LIFO) is a method used to account for inventory.
2. Under LIFO, the costs of the most recent products purchased (or produced) are the first to be expensed.
3. LIFO is used only in the United States and governed by the generally accepted accounting principles (GAAP).
3. Using LIFO typically lowers net income but is tax advantageous when prices are rising.

## Weighted Average
![image.png](Images/wa.jpg)
1. The weighted average takes into account the relative importance or frequency of some factors in a data set.
2. A weighted average is sometimes more accurate than a simple average.
3. Stock investors use a weighted average to track the cost basis of shares bought at varying times.
4. Most "Accurate" of the main 3

## E&O Inventory
![image.png](Images/eo.jpg)
1. Obsolete inventory is inventory at the end of its product life cycle that needs to be either written-down or written-off the company's books.
2. Obsolete inventory is written-down by debiting expenses and crediting a contra asset account, such as allowance for obsolete inventory.
3. The contra asset account is netted against the full inventory asset account to arrive at the current market value or book value.
4. When obsolete inventory is disposed of, both the related amount in the inventory asset account and the contra asset account are removed in the disposal journal entry.


## Net Working Capital & Operating Cash Flow - The Indirect Method
![image.png](Images/nwc.jpg)
1. Under the indirect method, the cash flow statement begins with net income on an accrual basis and subsequently adds and subtracts non-cash items to reconcile to actual cash flows from operations.
2. The indirect method is often easier to use than the direct method since most larger businesses already use accrual accounting.
3. The complexity and time required to list every cash disbursement—as required by the direct method—makes the indirect method preferred and more commonly used.
4. Increases (Decreases) in non-cash assets decrease (increase) cash flow
5. Increases (Decreases) in liabilities increase (decrease) cash flow

<a id='3'></a>
##  Data Manipulation 

In [ ]:
#Next, let's restructure the demand data so that we can get a weekly forecast from the monthly one.  We will
#assume a standard 4-4-5 calendar.

week_map = pd.read_csv('Week_Mapper.csv')



master = pd.DataFrame()
for i in range(1,13):
    temp = demand_data[demand_data['Month'] == i]
    temp = temp.merge(week_map, on = ['Month'], how = 'outer')
    temp = temp.drop(columns = 'Quarter')
    temp = temp.dropna()
    master = master.append(temp)

    
master['Demand'] = master['Demand'] / master['Weeks in Month']
scrubbed_demand = pd.DataFrame(columns = ['SKU_ID','Warehouse','Product_Family','Year'])
for i in range(1,53):
    temp = master[master['Week']==i]
    temp = temp.drop(columns = ['Week','Month','Weeks in Month','Unique_ID'])
    temp = temp.rename(columns = {'Demand':'Week_' + str(i) + '_Demand'})
    scrubbed_demand = scrubbed_demand.merge(temp,on = ['SKU_ID','Warehouse','Product_Family','Year'],how = 'outer')
    

scrubbed_demand.head()

<a id='4'></a>
## A note on Weeks of Supply
When measuring a company's inventory, it is often helpful to look at both absolte measures of inventory (such as number of units) as well as relative measures of inventory such as weeks of supply.  In general, weeks of supply is calculated as inventory in week 0 divided by the average of the demand for that inventory in weeks 1 ,2 ,3 , & 4.  This relative measure will allow us to compare relative efficient use of various products as well as areas that need attention.  

In [ ]:
#Merge in initial inventory positions.

scrubbed_demand = scrubbed_demand.merge(inventory_data, on = ['Warehouse','SKU_ID','Product_Family'], how = 'left')
scrubbed_demand.head()



In [ ]:
# Add in week 1 estimate of weeks of supply
scrubbed_demand['Week_1_WOS'] =  scrubbed_demand['Inventory as of 1/1/22'] / scrubbed_demand[
['Week_2_Demand','Week_3_Demand','Week_4_Demand','Week_5_Demand']].mean(axis = 1)
scrubbed_demand = scrubbed_demand.rename(columns = {'Inventory as of 1/1/22':'Week_0_Inventory'})
scrubbed_demand.to_csv('test.csv')

Weeks of supply tells the inventory manager how long the current on hand will last based on current sales demand.  By keeping your eye on weeks of supply, you can avoid inventory stock outs and lost sales.  The basic calculation for weeks of supply is pretty simple: on hand inventory / average weekly units sold.

<a id='5'></a>
## A note on production planning
As mentioned earlier, we will make the assumption that there is no supply chain chrisis that will prevent us from purchases the necessary raw materials to manufacture our goods, BUT, we don't want to buy so much that we risk our inventory expiring.  Weeks of supply is a great guage to see if we have enough or too much of any one item.  We will assume that we want to have 8 weeks of supply of each item we have on the books.  

Note that in the below cell, there is a recursive calculaiton occurring.  This is because each of our components is dependent on the other in some respect.  Weeks of supply depends on current demand and inventory, inventory depends on production and demand, and production depends on weeks of supply and demand.  

In [ ]:
wos_target = 8
for i in range(1,53):
    try:
        scrubbed_demand['Week_'+str(i)+'_Prod'] = np.where(scrubbed_demand['Week_'+str(i)+'_WOS'] >= wos_target , 
                                                           0,((wos_target+2) - scrubbed_demand['Week_'+str(i)+'_WOS']) * 
                                                           scrubbed_demand[['Week_'+str(i+1)+'_Demand','Week_'+str(i+2)+'_Demand','Week_'+str(i+3)+'_Demand','Week_'+str(i+4)+'_Demand']].mean(axis = 1))

        scrubbed_demand['Week_'+str(i)+'_Inventory'] = scrubbed_demand['Week_'+str(i-1)+'_Inventory'] - scrubbed_demand['Week_'+str(i)+'Demand'] + scrubbed_demand['Week_'+str(i)+'_Prod']
        
        scrubbed_demand['Week_'+str(i+1)+'_WOS'] = scrubbed_demand['Week_'+str(i)+'_Inventory']/ scrubbed_demand[['Week_'+str(i+2)+'_Demand','Week_'+str(i+3)+'_Demand','Week_'+str(i+4)+'_Demand','Week_'+str(i+5)+'_Demand']].mean(axis = 1)
    except:
        try:
            
            scrubbed_demand['Week_'+str(i)+'_Prod'] = np.where(scrubbed_demand['Week_'+str(i)+'_WOS'] >= wos_target , 
                                                           0,((wos_target+2) - scrubbed_demand['Week_'+str(i)+'_WOS']) * 
                                                           scrubbed_demand[['Week_'+str(i+1)+'_Demand','Week_'+str(i+2)+'_Demand','Week_'+str(i+3)+'_Demand']].mean(axis = 1))

            scrubbed_demand['Week_'+str(i)+'_Inventory'] = scrubbed_demand['Week_'+str(i-1)+
                                                                      '_Inventory'] - scrubbed_demand['Week_'+str(i)+'_Demand'] + scrubbed_demand['Week_'+str(i)+'_Prod']
            scrubbed_demand['Week_'+str(i+1)+'_WOS'] = scrubbed_demand['Week_'+str(i)+'_Inventory']/ scrubbed_demand[['Week_'+str(i+2)+'_Demand','Week_'+str(i+3)+'_Demand','Week_'+str(i+4)+'_Demand']].mean(axis = 1)
        except:
            try:
                scrubbed_demand['Week_'+str(i)+'_Prod'] = np.where(scrubbed_demand['Week_'+str(i)+'_WOS'] >= wos_target , 
                                                           0,((wos_target+2) - scrubbed_demand['Week_'+str(i)+'_WOS']) * 
                                                           scrubbed_demand[['Week_'+str(i+1)+'_Demand','Week_'+str(i+2)+'_Demand']].mean(axis = 1))
                scrubbed_demand['Week_'+str(i)+'_Inventory'] = scrubbed_demand['Week_'+str(i-1)+
                                                                      '_Inventory'] - scrubbed_demand['Week_'+str(i)+'_Demand'] + scrubbed_demand['Week_'+str(i)+'_Prod']
                scrubbed_demand['Week_'+str(i+1)+'_WOS'] = scrubbed_demand['Week_'+str(i)+'_Inventory']/ scrubbed_demand[['Week_'+str(i+2)+'_Demand','Week_'+str(i+3)+'_Demand']].mean(axis = 1)
            except:
                try:
                    scrubbed_demand['Week_'+str(i)+'_Prod'] = np.where(scrubbed_demand['Week_'+str(i)+'_WOS'] >= wos_target , 
                                                           0,((wos_target+2) - scrubbed_demand['Week_'+str(i)+'_WOS']) * 
                                                           scrubbed_demand[['Week_'+str(i+1)+'_Demand']].mean(axis = 1))
                    scrubbed_demand['Week_'+str(i)+'_Inventory'] = scrubbed_demand['Week_'+str(i-1)+
                                                                      '_Inventory'] - scrubbed_demand['Week_'+str(i)+'_Demand'] + scrubbed_demand['Week_'+str(i)+'_Prod']
                    scrubbed_demand['Week_'+str(i+1)+'_WOS'] = scrubbed_demand['Week_'+str(i)+'_Inventory']/ scrubbed_demand[['Week_'+str(i)+'_Demand']].mean(axis = 1)
                except:
                    try:
                        scrubbed_demand['Week_'+str(i)+'_Prod'] = np.where(scrubbed_demand['Week_'+str(i)+'_WOS'] >= wos_target , 
                                                           0,((wos_target+2) - scrubbed_demand['Week_'+str(i)+'_WOS']) * 
                                                           scrubbed_demand[['Week_'+str(i)+'_Demand']].mean(axis = 1))
                        scrubbed_demand['Week_'+str(i)+'_Inventory'] = scrubbed_demand['Week_'+str(i-1)+
                                                                      '_Inventory'] - scrubbed_demand['Week_'+str(i)+'_Demand'] + scrubbed_demand['Week_'+str(i)+'_Prod']
                    except:
                        print('Error')


In [ ]:
#Let's verticalize the data so it will be easier to visualize
final = pd.DataFrame()
for i in range(1,53):
    temp = scrubbed_demand[['Warehouse','Product_Family','Cost','SKU_ID','Year']]
    temp['Demand'] = scrubbed_demand['Week_' +str(i)+'_Demand']
    temp['WOS'] = scrubbed_demand['Week_' +str(i)+'_WOS']
    temp['Prod'] = scrubbed_demand['Week_' +str(i)+'_Prod']
    temp['Inventory'] = scrubbed_demand['Week_' +str(i)+'_Inventory']
    temp['Week'] = i
    final = final.append(temp)
temp = scrubbed_demand[['Warehouse','Product_Family','Cost','SKU_ID','Year']]
temp['Inventory'] = scrubbed_demand['Week_' +'0'+'_Inventory']
temp['Week'] = 0
final = final.append(temp)
final = final.merge(week_map, on = ['Week'], how = 'left')
final.to_csv('Forecast_Output.csv')
final.head()

<a id='6'></a>
## Preliminary Results & Commentary
We now have a raw view of our forecest for 2022 inventory, production and weeks of supply.   In the next step we will move to Tableau to see some effective ways to present the information as well as some provide color to what the key takeaways are for this exercise.  Please review the "Forecast_Output.csv" to review the structure of the data and make it easier to follow along.

Tableau Link:
https://public.tableau.com/app/profile/benjamin.smith4309/viz/Inventory_Mgmt/_Summary

<a id='7'></a>
## Common Inventory Issues that may throw a wrench in things
    1. Inconsistent Tracking:
Using manual inventory tracking procedures across different software and spreadsheets is time-consuming, redundant and vulnerable to errors. Even small businesses can benefit from a centralized inventory tracking system that includes accounting features.
    2. Warehouse Efficiency:
Inventory management controls at the warehouse is labor-intensive and involves several steps, including receiving and putaway, picking, packing and shipping. The challenge is to perform all these tasks in the most efficient way possible.
    3. Inaccurate Data:
You need to know, at any given moment, exactly what inventory you have. Gone are the days when inventory could be counted once a year with an all-hands-on-deck approach.
    4. Changing Demand:
Customer demand is constantly shifting. Keeping too much could result in obsolete inventory you’re unable to sell, while keeping too little could leave you unable to fulfill customer orders. Order strategies for core items, as well as technology to create and execute an inventory plan, can help compensate for changing demand.
    5. Limited Visibility:
When your inventory is hard to identify or locate in the warehouse, it leads to incomplete, inaccurate or delayed shipments. Receiving and finding the right stock is vital to efficient warehouse operations and positive customer experiences.
    6. Manual Documentation:
Managing inventory with paperwork and manual processes is tedious and not secure. And it doesn’t easily scale across multiple warehouses with lots of stock.
    7. Problem Stock:
Perishable and fragile stock need specialized plans for care and storage. And high-value inventory needs specific loss-prevention strategies and inventory controls.
    8. Supply Chain Complexity:
Global supply chains shift daily, placing a burden on your inventory planning and management operations. The manufacturers and wholesale distributors that dictate when, where and how your inventory ships require flexibility and offer unpredictable lead times.
    9. Managing Warehouse Space:
Efficiently managing space is an intimidating task. Planning and designing warehouse spaces with inventory management platforms helps you better control the timing of new stock deliveries. It can account for important factors, such as available space. Read more about the differences between warehouse management and inventory management.
    10. Insufficient Order Management:
One of the most common challenges to sound inventory management is preventing the overselling of products and running out of inventory. Using historical and seasonal data trends can help you accurately predict customer orders.
    11. Increasing Competition:
Globalized supply chains are subject to unpredictable economic shifts and market forces that impact the competition for raw materials. Small businesses are sometimes faced with choosing between competing for high-demand materials or holding enough inventory to control costs.
    12. Evolving Packaging:
Compostable packaging—or removing packaging all together—to reduce waste presents new obstacles for warehouse design and storage. It may even mean new equipment or shorter shelf life for some items.
    13. Expanding Product Portfolios:
Many online retail strategies remove the need for large warehouse distribution centers. These strategies make it easier to expand inventory and diversify product portfolios, but demand technology and resources for ordering, shipping and tracking.
    14. Overstocking:
Keeping too much stock on hand can be as problematic as having too little. Overstock impacts business cash flow and leads to inventory-related problems, such as storage and loss.
    15. Inventory Loss:
The loss of inventory due to spoilage, damage or theft can be a supply chain problem. It requires identifying, tracking and measuring problem areas.
    16. Poor Production Planning:
Production planning is vital for avoiding delayed manufacturing and cost overruns. If not done well, it can impact sales forecasts and project scheduling.
    17. Lack of Expertise:
It can be tough to find skilled inventory managers who are adept at the latest technology and can improve inventory strategy. Simply upgrading your inventory management platform with a host of features isn’t enough. You need capable management.
    18. Poor Communication:
Communication and collaboration are key. When departments are apathetic about sharing information, it makes identifying inventory trends and finding ways to improve much more difficult.
    19. Inefficient Processes:
Low-tech, manual inventory management procedures don’t seem like a daunting challenge when inventory is small and there’s only one warehouse location to manage. But as sales volume increases and inventory expands, inefficient, labor-intensive and low-tech standard operating procedures are difficult to scale.
    20. Inadequate Software:
To scale inventory management software to support complex logistics, it needs to integrate with your existing business process platforms. The difficult task is choosing from hundreds of inventory management solutions and mastering a host of features that require training and ongoing support. 